# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de preprocesamiento de observaciones de eBird
Se realizan virificaciones antes de obtener el dataframe final.

La salida de este script será un dataframe de pandas donde cada fila representará una observación de eBird.

El dataframe final tendrá las siguientes **columnas**: id_obs, id_celda, especie, familia, orden, fecha

## Importaciones

In [4]:
import pandas as pd
import sys
from pathlib import Path
from IPython.display import display

# Modulos propios
sys.path.append(str(Path('..')/'src'))
import asociar_grilla

## Carga de datos

**Nota para el usuario del notebook:** EL archivo "obs_ebird.txt" pesa 166 mb, por lo que es muy pesado para subirlo a GitHub. El mismo puede ser descargado de [google drive](https://drive.google.com/file/d/1Dlc4CDrUaHSdlqO_qWHAMROhHPoJIOhG/view?usp=sharing).

In [5]:
archivo = '../data/external/obs_ebird.txt'
try:
    df_base = pd.read_csv(archivo, sep='\t')
    display(df_base.head())
except FileNotFoundError:
    print('No se encuentra el archivo en la ruta ' + archivo)

C:\Users\pablo\AppData\Local\Temp\ipykernel_18744\3404752771.py:3: DtypeWarning: Columns (11,12,13,14,33,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv(archivo, sep='\t')


,GLOBAL UNIQUE IDENTIFIER,LAST EDITED DATE,TAXONOMIC ORDER,CATEGORY,TAXON CONCEPT ID,COMMON NAME,SCIENTIFIC NAME,SUBSPECIES COMMON NAME,SUBSPECIES SCIENTIFIC NAME,EXOTIC CODE,...,NUMBER OBSERVERS,ALL SPECIES REPORTED,GROUP IDENTIFIER,HAS MEDIA,APPROVED,REVIEWED,REASON,CHECKLIST COMMENTS,SPECIES COMMENTS,Unnamed: 52
0,URN:CornellLabOfOrnithology:EBIRD:OBS1966163490,2025-03-19 15:41:50.266885,6913,species,avibase-1819ECCE,Light-mantled Albatross,Phoebetria palpebrata,NaN,NaN,NaN,...,1.0,0,NaN,0,1,0,NaN,NaN,NaN,NaN
1,URN:CornellLabOfOrnithology:EBIRD:OBS1966176209,2025-03-19 15:41:50.266885,7354,species,avibase-1EA575E0,Magellanic Cormorant,Leucocarbo magellanicus,NaN,NaN,NaN,...,1.0,0,NaN,0,1,0,NaN,NaN,NaN,NaN
2,URN:CornellLabOfOrnithology:EBIRD:OBS2414496199,2025-03-24 07:15:47.007622,16879,species,avibase-644B907D,Austral Negrito,Lessonia rufa,NaN,NaN,NaN,...,4.0,1,NaN,0,1,0,NaN,NaN,NaN,NaN
3,URN:CornellLabOfOrnithology:EBIRD:OBS2414496193,2025-03-24 07:15:47.007622,28360,species,avibase-F09C3940,Austral Thrush,Turdus falcklandii,NaN,NaN,NaN,...,4.0,1,NaN,0,1,0,NaN,NaN,NaN,NaN
4,URN:CornellLabOfOrnithology:EBIRD:OBS2414496194,2025-03-24 07:15:47.007622,32378,species,avibase-AE31F0F3,Black-chinned Siskin,Spinus barbatus,NaN,NaN,NaN,...,4.0,1,NaN,0,1,0,NaN,NaN,NaN,NaN


## Limpieza de datos

### Revisión y eliminación de columnas no útiles

In [6]:
# Conservo únicamente las columnas necesarias y renombro según el formato de iNaturalist
df_filtrado = df_base[['COMMON NAME', 'SCIENTIFIC NAME', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']].rename(columns={
    'COMMON NAME': 'common_name',
    'SCIENTIFIC NAME': 'scientific_name',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'OBSERVATION DATE': 'observed_on'
})


##### Ver resumen de valores

In [7]:
# Ver la cantidad de datos faltantes en cada fila
print(df_filtrado.isnull().sum())

common_name        0
scientific_name    0
latitude           0
longitude          0
observed_on        0
dtype: int64


In [8]:
# Reviso puntos extremos de las coordenadas
df_filtrado.describe()

,latitude,longitude
count,426837.000000,426837.000000
mean,-54.551502,-67.848963
std,0.525515,1.071940
min,-57.920581,-68.774447
25%,-54.837786,-68.350963
50%,-54.811116,-68.268240
75%,-54.185825,-67.723954
max,-52.454670,-58.636610


## Asociar la celda de la grilla a cada observación

In [9]:
df_filtrado

,common_name,scientific_name,latitude,longitude,observed_on
0,Light-mantled Albatross,Phoebetria palpebrata,-54.773000,-64.289333,1900-02-01
1,Magellanic Cormorant,Leucocarbo magellanicus,-54.773000,-64.289333,1900-02-01
2,Austral Negrito,Lessonia rufa,-54.810974,-68.321228,1972-12-10
3,Austral Thrush,Turdus falcklandii,-54.810974,-68.321228,1972-12-10
4,Black-chinned Siskin,Spinus barbatus,-54.810974,-68.321228,1972-12-10
...,...,...,...,...,...
426832,Yellow-billed Teal,Anas flavirostris,-54.837786,-68.350963,2025-04-02
426833,Yellow-billed Teal,Anas flavirostris,-54.837786,-68.350963,2025-04-15
426834,Yellow-billed Teal,Anas flavirostris,-54.817429,-68.318503,2025-04-07
426835,Yellow-billed Teal,Anas flavirostris,-54.817429,-68.318503,2025-04-03


In [13]:
# Punto de partida: directorio actual (notebooks/)
BASE_DIR = Path.cwd()
# Subir un nivel al directorio raíz del proyecto
PROYECTO_DIR = BASE_DIR.parent
# Construir ruta completa al archivo GPKG
DIR_DATOS = PROYECTO_DIR / 'data' / 'raw'

nombre_grilla = 'grilla_tdf_vacia.gpkg'
ruta_grilla = DIR_DATOS / nombre_grilla

In [14]:
# Asociar grilla a los datos de observaciones

df_grillada =asociar_grilla.assign_grid_cell_ids(
    grilla = ruta_grilla,
    datos_georef = df_filtrado,
    grid_id_field = 'grid_id'
)

## Exportar el data frame resultante

In [15]:
df_grillada.to_csv('../data/raw/data_ebird.csv', index=False)